## Import libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from os import path
#%matplotlib notebook

from brainpipe.system import study
from brainpipe.feature import TF
from brainpipe.visual import *
from brainpipe.statistics import *

In [2]:
sf = 512
n_perm = 1000
f = (1,120,4,2)  # Frequency vector: (from, to, width, step) 1 to 120 Hz 4 2 #0.1, 13, 0.5, 0.1
lines = [975,4000]
baseline = [0, 700] # Time in seconds (rest 700ms au milieu des 1s extraite)
width, step = 50, 10 #Time vector

### Compute TF on unbalanced data for all selected electrodes
#### No baseline correction - over 5.5s after odor is sent

In [ ]:
st = study('Olfacto')
path_data = path.join (st.path, 'database/Test_Good_Bad_EpiScore_Expi/')
path2save = path.join (st.path, 'feature/1_TF_Test/')
conds = ['bad', 'good']
#subjects = ['LEFC','CHAF','VACJ','SEMC','FERJ','MICP','PIRJ']
subjects = ['LEFC','CHAF','VACJ','SEMC','FERJ','MICP','PIRJ']

for su in subjects:
    bad_data = np.load(path.join(path_data, su+'_odor_bad_bipo_sel.npz'))['x']
    good_data = np.load(path.join(path_data, su+'_odor_good_bipo_sel.npz'))['x']
    names = np.load(path.join(path_data, su+'_odor_good_bipo_sel.npz'))['label']
    channels = np.load(path.join(path_data, su+'_odor_good_bipo_sel.npz'))['channel']
    print (su, 'bad shape: ', bad_data.shape, 'good shape: ', good_data.shape)

# ========================= COMPUTE TF FOR 1 ELEC =============================================
    for elec in range(bad_data.shape[0]):
        bad, good = bad_data[elec,:,:], good_data[elec,:,:]
        channel, label, sf = channels[elec], names[elec], 512
        print (su, channel, label, bad.shape, good.shape, 'nb points', bad.shape[0])
        
        #Get the TF
        # time = all data points in the original data (window to plot is selected afterwards)
        time = 1000 * np.arange(bad.shape[0]) / sf #to set the 0 in your TF
        print ('Time points: ', len(time),min(time), max(time))
        tfObj = TF(sf, bad.shape[0], f=f, time=time, width=width, step=step,baseline=baseline, norm=3)
        xtf_bad, _ = tfObj.get((bad), statmeth=None, n_perm=n_perm, maxstat=False, tail=2, metric='m_center')
        xtf_good, _ = tfObj.get((good), statmeth=None, n_perm=n_perm, maxstat=False, tail=2, metric='m_center')
        print ('TF size :', xtf_bad.shape, xtf_good.shape)
        xtf_bad, xtf_good = 100*np.swapaxes(xtf_bad,0,1), 100*np.swapaxes(xtf_good,0,1)
        
        # plot both TF in the same figure
        xtf_all = np.concatenate((xtf_bad, xtf_good), axis=0)
        print ('TF size :', xtf_bad.shape,xtf_good.shape)
        fig = plt.figure(elec, figsize=(10, 4))
        title = su+' Elec('+str(elec)+') '+channel+' '+label+' Bad/Good'
        fig, allax = tfObj.plot2D(fig, xtf_all, cmap='viridis', xvec=tfObj.xvec,
                     yvec=tfObj.yvec, xlabel='Time (ms)', vmin=-100, vmax=100, ycb=-20,
                     ylabel='Frequency (hz)', figtitle=title, title=['Bad Odors', 'Good odors'],
                     cblabel='Power modulations (%)', pltype='imshow', resample=(0.1, 0.1),
                    sharex=False, sharey=False, subdim=(1,2), subspace={'top':0.8})
        for k in allax:
            addLines(k, vLines=lines, vColor=['firebrick']*2, vWidth=[2]*2)
        #Save all your plots
        fname = (path2save+su+'_TF_elec_('+str(elec)+')_'+channel+'_'+label+'_Bad-Good.png')
        print (fname)
        fig.savefig(fname, dpi=300, bbox_inches='tight')
        plt.close()
    del bad, good, sf, channel

-> Olfacto loaded
LEFC bad shape:  (139, 2816, 23) good shape:  (139, 2816, 35)
LEFC a2-a1 Amg-pPirT (2816, 23) (2816, 35) nb points 2816
Time points:  2816 0.0 5498.046875
TF size : (58, 1, 277) (58, 1, 277)
TF size : (1, 58, 277) (1, 58, 277)
/media/karim/Datas4To/1_Analyses_Intra_EM_Odor/Olfacto/feature/1_TF_Test/LEFC_TF_elec_(0)_a2-a1_Amg-pPirT_Bad-Good.png
LEFC a6-a5 Ins-Ppo (2816, 23) (2816, 35) nb points 2816
Time points:  2816 0.0 5498.046875
TF size : (58, 1, 277) (58, 1, 277)
TF size : (1, 58, 277) (1, 58, 277)
/media/karim/Datas4To/1_Analyses_Intra_EM_Odor/Olfacto/feature/1_TF_Test/LEFC_TF_elec_(1)_a6-a5_Ins-Ppo_Bad-Good.png
LEFC a7-a6 Ins-Ppo (2816, 23) (2816, 35) nb points 2816
Time points:  2816 0.0 5498.046875
TF size : (58, 1, 277) (58, 1, 277)
TF size : (1, 58, 277) (1, 58, 277)
/media/karim/Datas4To/1_Analyses_Intra_EM_Odor/Olfacto/feature/1_TF_Test/LEFC_TF_elec_(2)_a7-a6_Ins-Ppo_Bad-Good.png
LEFC a8-a7 aSTG-aMTG&Ins-Ppo (2816, 23) (2816, 35) nb points 2816
Time point

## TF by cond - Good & Bad Odors
### Group both conditions later with an other script (Tools)

In [ ]:
sf = 512
n_perm = 1000
f = (0.1, 13, 0.5,0.25)  # Frequency vector: (from, to, width, step) 1 to 120 Hz 4 2 #0.1, 13, 0.5, 0.1
bsl = 'Rest'
if bsl == 'Rest':
    baseline = [150, 850] # Time in seconds (rest 700ms au milieu des 1s extraite)
elif bsl == 'Exp':
    baseline = [1300, 2000] # Time in seconds (rest 500ms au milieu des 1s extraite)
lines = [0]
norm = 3
width, step = 50, 10 #Time vector
window_to_plot = [-500, 2000] # In seconds

In [ ]:
st = study('Olfacto')
path_data = path.join (st.path, 'database/TS_E_all_by_odor_th40_art400_30_250_Good_Bad_EpiScore_Expi/')
path2save = path.join (st.path, 'TimeFrequency/TF_Encoding_all_filter1_th40_400art_30_250_GoodBad_EPISCORE_Rois/')
conds = ['bad', 'good']

test = False
if test == True:
    n_elec = {'PIRJ' :1}
    subjects = ['PIRJ']
else :
    subjects = ['VACJ','SEMC','PIRJ','LEFC','MICP','FERJ']#'CHAF',
    n_elec = {
    'CHAF' : 69,
    'VACJ' : 84, 
    'SEMC' : 66,
    'PIRJ' : 71,
    'LEFC' : 152,
    'MICP' : 76,
    'FERJ' : 88,
        }

for su in subjects:
    bad_data = np.load(path.join(path_data, su+'_concat_odor_bad_bipo_new.npz'))['x']
    good_data = np.load(path.join(path_data, su+'_concat_odor_good_bipo_new.npz'))['x']
    names = np.load(path.join(path_data, su+'_concat_odor_good_bipo_new.npz'))['label']
    channels = np.load(path.join(path_data, su+'_concat_odor_good_bipo_new.npz'))['channel']
    print (su, 'bad shape: ', bad_data.shape, 'good shape: ', good_data.shape)

# ========================== SELECT ONLY ELECTRODES IN MTL ===================================
    elecs_sel = []
    for roi in rois:
        for elec in range(n_elec[su]):
            elec_sel = np.where(names[elec].find(roi) != -1,int(elec),None)
            if elec_sel != None:
                elecs_sel = np.append(elecs_sel,elec_sel)
    # remove duplicates in elec list
    elecs_final=[]
    for i in elecs_sel:
        if i not in elecs_final:
            elecs_final.append(i)
    print('sel', elecs_sel,len(elecs_sel), 'final',elecs_final, len(elecs_final))

# ==========================  BALANCED CONDITIONS ===========================================
    if bad_data.shape[2] > good_data.shape[2]:
        bad_stat = bad_data[:,:,np.random.randint(bad_data.shape[2], size=good_data.shape[2])]
        good_stat = good_data
    elif bad_data.shape[2] < good_data.shape[2]:
        bad_stat = bad_data
        good_stat = good_data[:,:,np.random.randint(good_data.shape[2], size=bad_data.shape[2])]
    else:
        bad_stat, good_stat = bad_data, good_data
    ntrials = bad_stat.shape[2]
    print ('balanced data : ', bad_stat.shape, good_stat.shape)

# ========================= COMPUTE TF FOR 1 ELEC =============================================
    for elec in elecs_final:
        bad, good = bad_stat[elec,:,:], good_stat[elec,:,:]
        channel, label, sf = channels[elec], names[elec], 512
        print (su, channel, label, bad.shape, good.shape, 'nb points', bad.shape[0])
        
        #Get the TF
        # time = all data points in the original data (window to plot is selected afterwards)
        time = 1000 * np.arange(-1024, bad.shape[0]-1024) / sf #to set the 0 in your TF
        print ('Time points: ', len(time),min(time), max(time))
        tfObj = TF(sf, bad.shape[0], f=f, baseline=baseline,norm=norm, time=time, width=width, step=step)
        xtf_bad, _ = tfObj.get((bad), statmeth=None, n_perm=n_perm, maxstat=False, tail=2)
        xtf_good, _ = tfObj.get((good), statmeth=None, n_perm=n_perm, maxstat=False, tail=2)
        print ('TF size :', xtf_bad.shape, xtf_good.shape)
        xtf_bad, xtf_good = 100*np.swapaxes(xtf_bad,0,1), 100*np.swapaxes(xtf_good,0,1)
        xtf_all = np.concatenate((xtf_bad, xtf_good), axis=0)
        print ('TF size :', xtf_bad.shape,xtf_good.shape) #'pvaltf : ', pvaltf.shape, pvaltf) #Freq bands * time windows

        # Plot everything that is inside the window parameter :
        timebin = np.array(tfObj.xvec)
        sl = slice(np.argmin(np.abs(timebin-window_to_plot[0])), np.argmin(np.abs(timebin-window_to_plot[1])))

        # Finally plot your TF
        fig = plt.figure(elec, figsize=(10, 4))
        title = su+' Elec('+str(elec)+') '+channel+' '+label+' Bad/Good ntrials:'+str(ntrials)
        fig, allax = tfObj.plot2D(fig, xtf_all[:,:, sl], cmap='viridis', xvec=tfObj.xvec[sl],
                     yvec=tfObj.yvec, xlabel='Time (ms)', vmin=-100, vmax=100, ycb=-20,
                     ylabel='Frequency (hz)', figtitle=title, title=['Bad Odors', 'Good odors'],
                     cblabel='Power modulations (%)', pltype='imshow', resample=(0.1, 0.1),
                    sharex=False, sharey=False, subdim=(1,2), subspace={'top':0.8})
        for k in allax:
            addLines(k, vLines=lines, vColor=['firebrick'], vWidth=[2])
        #Save all your plots
        fname = (path2save+su+'_TF_elec_('+str(elec)+')_'+channel+'_'+label+'_Bad-Good_'+bsl+'.png')
        print (fname)
        fig.savefig(fname, dpi=300, bbox_inches='tight')
        plt.close()
    del bad, good, sf, channel